In [ ]:
import pickle
import random

import glob
import math


def get_traces(path):
    files = glob.glob(path)

    traces = []
    for file in files:
        with open(file, "rb") as f:
            trace = pickle.load(f)
            traces.append(trace)
    return traces

traces = get_traces('../traces_2023-10-30_12:18/*')

In [ ]:
from refactor.proof_node import InternalNode, ErrorNode, ProofFinishedNode

In [ ]:
def add_data(node, visits):
    data = []
    # proving_edges = [edge for edge in node.out_edges if edge.distance_to_proof() < math.inf]
    # for edge in proving_edges:
    if node.out_edges:
        for edge in node.out_edges:
            datum = {
                'goal': node.goal,
                'tactic': edge.tactic,
                'distance_to_proof': edge.distance_to_proof(),
                'visits': visits[node.goal],
                'time': edge.time,
                'logprob': edge.logprob
            }
            # add children of edge
            if len(edge.dst) == 1 and isinstance(edge.dst[0], ErrorNode):
                # todo could record error message for e.g. self-correcting proof approach>
                datum['result'] = ['Error']
            elif len(edge.dst) == 1 and isinstance(edge.dst[0], ProofFinishedNode):
                datum['result'] = ['Proven']
            else:
                result = [d.goal for d in edge.dst]
                datum['result'] = result
            data.append(datum)
    return data


full_data = []

for trace in traces:
    nodes = trace.nodes

    updated_visit_count = {node: nodes[node].visit_count for node in nodes}

    for goal, node in nodes.items():
        for a in node.ancestors:
            updated_visit_count[a] += node.visit_count

    for node in nodes:
        full_data.extend(add_data(nodes[node], updated_visit_count))


In [ ]:
# todo
def add_goal_data(node, visits):
    steps = node.distance_to_proof


    datum = {
        'goal': node.goal,
        'distance_to_proof': node.distance_to_proof,
        'visits': visits[node.goal],
        'up_score': node.up_score,
        'local_visits': node.visit_count
    }


    if steps == math.inf:
        if visits[node.goal] >= neg_threshold:
            target = 0
        else:
            return None
    elif steps >= max_proof_len:
        target = 1
    else:
        # linearly project steps so that shorter proofs are allocated to higher buckets
        # target = (max_bucket + 1 - steps) // (max_bucket // buckets)
        target = (max_proof_len + 1) - steps

    return {'goal': node.goal, 'target': target}




goal_step_data = []

for trace in traces:
    nodes = trace.nodes

    updated_visit_count = {node: nodes[node].visit_count for node in nodes}

    for goal, node in nodes.items():
        for a in node.ancestors:
            updated_visit_count[a] += node.visit_count

    for node in nodes:
        step_datum = add_goal_step_data(nodes[node], updated_visit_count)
        if step_datum:
            goal_step_data.append(step_datum)


In [ ]:
# todo search traces

In [ ]:
# todo save all to mongodb collections


# todo wandb/hydra/resuming eval

# todo train scripts for eval models

# todo htps

# todo add BFS, bestfs
